# Contenuti
- [Scikit-learn](#Scikit-learn)
- [Import delle librerie](#Import-delle-librerie)
- [Dataset](#Dataset)
- [Classificazione](#Classificazione)
    - [k-NN e SVM](#k-NN-e-SVM)
- [Cross-Validation](#Cross-Validation)
- [Grid Search e Cross-Validation](#Grid-Search-e-Cross-Validation)
- [Ottimizzazione iperparametri](#Ottimizzazione-iperparametri)
- [Test](#Test)

# Esercitazione 3
Nell'esercitazione odierna si vedrà come applicare gli algoritmi *k-NN* e *SVM* a un problema di classificazione multiclasse. A tal fine verrà utilizzata la libreria **Scikit-learn**, di cui sarà descritta l'API per l'addestramento e uso di classificatori. Nel corso dell'esercitazione si dovranno individuare le combinazioni di iperparametri che permettono di massimizzare l’accuratezza dei due classificatori sui dataset forniti tramite *Cross-Validation* e *Grid Search*.

Infine si dovrà verificare l'accuratezza della soluzione trovata sul dataset di test per provarne l’effettiva capacità di generalizzazione.

# Scikit-learn
Scikit-learn è una libreria open source per Python che mette a disposizione numerose classi e funzionalità specifiche per il Machine Learning. Oltre a rendere disponibili i più comuni algoritmi per la classificazione, regressione e clustering, sono presenti anche diverse funzionalità per la manipolazione dei dati.

La popolarità di Scikit-learn è dovuta principalmente alla sua facilità d'uso, all'integrazione con la libreria Numpy e alla possibilità di parametrizzare le operazioni messe a disposizione, pur mantenendo una API semplice.

Per una lista completa delle funzionalità si rimanda alla [documentazione ufficiale](http://scikit-learn.org/stable/documentation.html).

# Import delle librerie
Per prima cosa è necessario eseguire l'import delle librerie utilizzate durante l'esecitazione. Il package della libreria Scikit-learn è denominato **sklearn**. Dal package è possibile caricare solamente i moduli necessari.

In [ ]:
# Moduli di scikit-learn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, ParameterGrid, train_test_split

import ml_utilities
import ml_visualization

# Dataset
Il dataset messo a disposizione consiste in un insieme di cifre (0-9) scritte a mano.
<img src="dataset_resample.png" alt="Processing training set" style="width: 600px;"/>

I dataset di training "pendigits_tr.txt" e "pendigits_tr_Pca_K2.txt" contengono entrambi 7330 pattern etichettati (cifre da 0 a 9). Il primo contiene pattern 16-dimensionali (le coordinate x,y di otto punti equispaziati dopo normalizzazione e *resampling*) mentre il secondo consiste in una versione bidimensionale ottenuta tramite riduzione della dimensionalità (mediante l'algoritmo *Principal Component Analysis*), utile per eseguire test iniziali visualizzando pattern e risultati.

Di seguito è fornita la procedura per il caricamento del dataset. È necessario assegnare alla variabile *dataset_path* il percorso dove sono memorizzati i due file. Tramite la variabile *feature_count* è possibile dichiarare quale delle due versioni del dataset si intende utilizzare.

In [ ]:
# Scelta del dataset da utilizzare (16 o 2 features)
feature_count = 2
dataset_path = 'pendigits_tr_Pca_K2.txt'  # Impostare il percorso corretto

# Caricamento del dataset
dataset_patterns, dataset_labels = ml_utilities.load_labeled_dataset_from_txt(dataset_path, feature_count)
print('Shape dataset:', dataset_patterns.shape)
print('Shape labels:', dataset_labels.shape)

Il dataset caricato può essere suddiviso in due parti: training e validation set. Attraverso il validation set sarà possibile valutare i risultati del classificatore addestrato sul training set al fine di individuare il valore ottimale per gli iperparametri.

Visto che in Machine Learning è comune eseguire tale operazione, la libreria Scikit-learn mette a disposizione una apposita funzione, [**train_test_split(...)**](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html), che permette di separare un dataset in due parti. 

Il parametro *test_size* descrive la percentuale di pattern che dovrà essere contenuta nella seconda parte. Come configurazione predefinita **train_test_split(...)** mescola i pattern al fine di evitare che i dataset restituiti contengano pattern appartenenti solamente a un sottoinsieme delle classi.

In [ ]:
train_x, validation_x, train_y, validation_y = train_test_split(dataset_patterns, dataset_labels, test_size=0.40)
print('Shape training set:', train_x.shape)
print('Shape validation set:', validation_x.shape)

# Classificazione
Di seguito vengono descritte le operazioni necessarie per creare e addestrare un classificatore. Uno dei punti di forza della libreria Scikit-learn è la sua facilità d'uso e l'uniformità delle chiamate alla libreria: si noti come queste operazioni necessitino solamente di due linee di codice.

In [ ]:
# Creazione di un classificatore
dummy_clf = DummyClassifier(strategy='most_frequent')

Nell'esempio, il classificatore creato è un **DummyClassifier** la cui strategia è quella di assegnare ad ogni pattern la classe più numerosa nel training set.

Il classificatore (*estimator* nel lessico di Scikit-learn) può essere addestrato tramite il metodo **.fit(...)** che prende in input un insieme di pattern etichettati X,y. I pattern (X) sono memorizzati per righe in un Numpy array bidimensionale mentre le etichette (y) in un Numpy array monodimensionale.

N.B.: la notazione X (maiuscolo) e y (minuscolo) è normalmente utilizzata in Scikit-learn per indicare un set di pattern (X) e le rispettive etichette (y).

In [ ]:
# Addestramento
dummy_clf.fit(train_x, train_y)

Una volta addestrato, un classificatore può essere utilizzato per riconoscere la classe di uno o più pattern. Per fare ciò, si può utilizzare il metodo **.predict(...)** passando come parametro una lista di pattern. Il risultato che si ottiene è una lista delle label predette dal classificatore.

In [ ]:
# Uso della funzione .predict(...)
print('Classi predette:', dummy_clf.predict(validation_x[0:5]))
print('Classi reali:', validation_y[0:5])

In Scikit-learn è possibile misurare l'accuratezza di un classificatore su un determinato dataset richiamando il metodo **.score(...)** che prende in input un insieme di pattern etichettati X,y. Internamente il metodo classifica ogni singolo pattern di X e verifica se l'etichetta assegnata corrisponde alla corrispondente *entry* in y.

In [ ]:
# Uso della funzione .score(...)
print('Accuratezza sul training set: {}'.format(dummy_clf.score(train_x, train_y)))
print('Accuratezza sul validation set: {}'.format(dummy_clf.score(validation_x, validation_y)))

Si implementi la funzione **compute_accuracy(...)** in grado di calcolare l'accuratezza di un classificatore su un insieme di pattern etichettati X,y.

In [ ]:
# Esercizio di approfondimento: stima dell'accuratezza
def compute_accuracy(classifier, X, y):
    results = classifier.predict(X)
    return (results == y).sum() / len(X)

print('Accuratezza sul training set: {}'.format(compute_accuracy(dummy_clf, train_x, train_y)))
print('Accuratezza sul validation set: {}'.format(compute_accuracy(dummy_clf, validation_x, validation_y)))

Per dati bidimensionali può essere utile visualizzare graficamente i pattern e la superfice decisionale del classificatore. Per fare ciò si può utilizzare la funzione **show_2D_results(...)** presente nel modulo ml_visualization.

In [ ]:
# Uso di show_results
ml_visualization.show_2D_results(dummy_clf, (train_x, train_y, 'Training'), (validation_x, validation_y, 'Validation'))

## k-NN e SVM
Abbiamo visto come, con Scikit-learn, sia possibile creare, addestrare e utilizzare un generico classificatore. Scikit-learn mette a disposizione una serie di classificatori tra cui [k-NN](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) e [SVM](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) che utilizzeremo per riconoscere i pattern caricati in precedenza.

In [ ]:
# Esercizio 1: creare ed addestrare un classificatore

# In questo esercizio si richiede di creare 3 tipi di classificatore ...
# - k-NN
# - SVM con kernel lineare
# - SVM con kernel rbf
# A tal fine si chiede di prendere confidenza con la documentazione 
# identificando in essa gli iperparametri rilevanti per i diversi
# classificatori / kernel.

# Creazione del classificatore: soluzione
# Di seguito è riportata la procedura corretta per creare i suddetti classificatori.

clf = KNeighborsClassifier(n_neighbors=5)
# clf = SVC(kernel="linear", C=1)
# clf = SVC(kernel="rbf", C=1, gamma=0.1)

# Addestramento del classificatore: soluzione
clf.fit(train_x, train_y)

# Calcolo accuratezza
print('Accuratezza sul training set: {}'.format(clf.score(train_x, train_y)))
print('Accuratezza sul validation set: {}'.format(clf.score(validation_x, validation_y)))

#Visualizzazione 2D
ml_visualization.show_2D_results(clf, (train_x, train_y, 'Training'), (validation_x, validation_y, 'Validation'))

# Cross-Validation
Se non si vuole riservare un sottoinsieme disgiunto per il validation set e sfruttare tutti i dati disponibili per training e validazione, si può utilizzare la tecnica *Cross-Validation* vista a lezione.  

Si completi il codice nella cella seguente richiamando la funzione [**cross_val_score(...)**](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) di Scikit-learn che stima l'accuratezza di un classificatore (*estimator*) tramite *Cross-Validation* (con un numero di *fold* pari a *cv*).

In [ ]:
# Esercizio 2: eseguire la Cross Validation utilizzando la funzione cross_val_score
# Reminder: la variabile dataset_patterns include già training e validation set

cv_score = cross_val_score(KNeighborsClassifier(n_neighbors=5),
                           dataset_patterns,
                           dataset_labels,
                           cv=4)
print('Risultati CrossValidation:\n', cv_score)

# Grid Search e Cross-Validation
Dato un algoritmo, la ricerca degli iperparametri ottimi può essere automatizzata utilizzando, come visto a lezione, la classe [**GridSearchCV**](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) messa a disposizione da Scikit-learn.

Questo metodo è particolarmente indicato nei casi in cui l'addestramento di un classificatore non richieda un tempo particolarmente lungo. Si consideri infatti che vengono creati e addestrati $(\textit{# combinazioni degli iperparametri} \times \textit{# fold}) + 1$ classificatori.

In [ ]:
# Creazione di un classificatore di tipo SVM
clf = SVC()

# Creazione della griglia di iperparametri
param_grid = [{'kernel': ['rbf'], 'C': [1,10], 'gamma': [0.05,0.025]}]

# Numero di fold per la Cross-validation
n_folds = 3

# Creazione di un oggetto di tipo GridSearchCV
grid_search_cv = GridSearchCV(clf, param_grid, cv=n_folds)

# Esecuzione della ricerca degli iperparametri 
grid_search_cv.fit(train_x, train_y)

# Stampa risultati
print('Combinazioni di parametri:\n', grid_search_cv.cv_results_['params'])
print('Accuratezza media per combinazione:\n', grid_search_cv.cv_results_['mean_test_score'])
print('Combinazione migliore:\n', grid_search_cv.best_params_)
print('Accuratezza media della combinazione migliore: {}'.format(grid_search_cv.best_score_))

# Visualizzazione 2D
ml_visualization.show_2D_results(grid_search_cv.best_estimator_,
                                 (dataset_patterns, dataset_labels, 'Data'),
                                 figsize=(9,8))

Se si vuole testare esternamente al GridSearchCV(...) i migliori iperparametri precedentemente individuati (best_params_), si consiglia di verificarne la bontà utilizzando la funzione cross_val_score(...) che restituisce l'accuratezza per le singole fold. Viceversa, utilizzando l'intero training set sia per l'addestramento che per la valutazione si rischia di ottenere un valore di accuratezza non realistico.

In [ ]:
# Costruisce un classificatore con i parametri trovati
clf = SVC(**grid_search_cv.best_params_)
print('Classificatore con parametri ottimali:\n', clf, '\n')

# Cross validation
cv_score = cross_val_score(clf, train_x, train_y, cv=n_folds)
print('Accuracy di cross validation: ', cv_score)
print('Accuracy media di cross validation: ', cv_score.mean())

# Addestramento su tutto il dataset
clf = SVC(**grid_search_cv.best_params_)
clf.fit(train_x, train_y)
print('Accuracy classificatore addestrato su tutto il training set:', clf.score(train_x, train_y))

# Ottimizzazione iperparametri
Si individui il classificatore (*k-NN* o *SVM*) e la rispettiva combinazione di iperparametri che permettono di massimizzare l'accuratezza sul dataset fornito (16-dimensionale).

Si consiglia di procedere eseguendo **GridSearchCV** su un insieme limitato di combinazioni raffinando la ricerca degli iperparametri in iterazioni successive. In questa maniera sarà possibile ridurre i tempi di attesa.

Nella cella seguente è riportata la procedura per il caricamento del dataset:

In [ ]:
# Caricamento del dataset
feature_count = 16
dataset_path = 'pendigits_tr.txt'  # Impostare il percorso corretto

dataset_patterns, dataset_labels = ml_utilities.load_labeled_dataset_from_txt(dataset_path, feature_count)

Utilizzando il dataset appena caricato si identifichi una combinazione di iperparametri ottimi. A tal fine nella cella seguente è possibile sperimentare le tecniche viste in questa esercitazione.

In [ ]:
# Esercizio 3: ottimizzazione iperparametri

# La strategia migliore è quella di utilizzare GridSearchCV sull'intero dataset
# tentando diverse combinazioni di iperparametri un po' per volta (4-8 per volta)
# in maniera tale da poter ottenere velocemente i risultati.

# Per ottenere punteggi attendibili è necessario utilizzare un numero
# di cv (fold) compreso tra 3 e 10

# Creazione del classificatore e della griglia di iperparametri
clf = KNeighborsClassifier()
param_grid = [{'n_neighbors': range(1, 11)}]

#clf = SVC()
#param_grid = [{'kernel': ['linear'], 'C': [1, 5, 10]}]

#clf = SVC()
#param_grid = [{'kernel': ['rbf'], 'C': [1, 10], 'gamma': [0.1, 0.01]}]

# Numero di fold per la Cross-validation
n_folds = 3

# Creazione di un oggetto di tipo GridSearchCV
experiment_gscv = GridSearchCV(clf, param_grid, cv=n_folds)

# Esecuzione della ricerca degli iperparametri 
experiment_gscv.fit(dataset_patterns, dataset_labels)

# Stampa risultati
print('Combinazioni di parametri:\n', experiment_gscv.cv_results_['params'])
print('Accuratezza media per combinazione:\n', experiment_gscv.cv_results_['mean_test_score'])
print('Combinazione migliore:\n', experiment_gscv.best_params_)
print('Accuratezza media della combinazione migliore: {}'.format(experiment_gscv.best_score_))